In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os
import re

Using TensorFlow backend.


In [14]:
os.listdir(folder)[3]

'Stus_Vasyl.Tom_3_1.Palimpsesty.1576.ua.txt'

In [22]:
# let's try Stus poetry
def readfile(fname):
    with open(fname, 'r') as f:
        return f.read().lower()
    
def preprocess(poem):
    #poem = re.sub('\d+\.\s?|\s?\*\s?\*\s?(\*\s?)?|\[*\]|\<*\>', '', poem)
    # left only words
    poem = re.sub('[^!а-яіїєА-ЯІЇЄ\s\,\.\-\—\:\n\!\(\)\?’]', ' ', poem)
    return poem.replace('\t', '\n')

folder = './generation/stus/'
#raw_text = '|'.join([preprocess(readfile(folder + file)) for file in os.listdir(folder)])
raw_text = preprocess(readfile(folder + os.listdir(folder)[3]))

In [24]:
chars = set(raw_text)
char_to_int =  { c:i for i, c in enumerate(chars) }
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: {}".format(n_chars))
print("Total Vocab: {}".format(n_vocab))

Total Characters: 214548
Total Vocab: 44


In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: {}".format(n_patterns))

Total Patterns: 741268


In [8]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [9]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [11]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
 19200/741268 [..............................] - ETA: 9523s - loss: 3.4028

KeyboardInterrupt: 